#### Straight Line

In [ ]:
# Test the prompts
import json 
import random
from codesim.prompt import StraightLine as SL
%load_ext autoreload
%autoreload 2

# Get the data
data = json.load(open("./data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_name = random.choice(list(lab_syn.keys()))
gt = lab_syn[var_name]
question = SL.questions_syn[0].format(varname=var_name)
prompt_syn = SL.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_name = random.choice(list(lab_nat.keys()))
object_name = random.choice(list(lab_nat[agent_name].keys()))
gt = lab_nat[agent_name][object_name]
question = SL.questions_nat[0].format(varname=object_name, agentname=agent_name)
prompt_nat = SL.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Critical Path

In [ ]:
# Test the prompts
import json 
import random
from codesim.prompt import CriticalPath as CP
%reload_ext autoreload
%autoreload 2

# Get the data
data = json.load(open("./data/CriticalPath/n_ops-20_n_vars-4_len_critical_path-10_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_name = random.choice(list(lab_syn.keys()))
gt = lab_syn[var_name]
question = CP.questions_syn[0].format(varname=var_name)
prompt_syn = CP.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_name = random.choice(list(lab_nat.keys()))  # can sample here
gt = lab_nat[agent_name]
question = CP.questions_nat[0].format(agentname=agent_name)
prompt_nat = CP.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)


#### Parallel Paths

In [ ]:
# Test the prompts
import json 
from codesim.prompt import ParallelPaths as PP

# Get the data
data = json.load(open("./data/ParallelPaths/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
var_names = list(lab_syn.keys())
gt = list(lab_syn.values())
question = PP.questions_syn[0].format(varnames=", ".join(var_names))
prompt_syn = PP.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

print("="*30)
# Nat
agent_names = list(lab_nat.keys())
object_names = list(lab_nat[agent_names[0]].keys())
gt = list(lab_syn.values())
question = PP.questions_nat[0].format(varnames=object_names,
                                      agentnames=agent_names)
prompt_nat = PP.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Sort

In [ ]:
# Test the prompts
import json 
from codesim.prompt import Sort

# Get the data
data = json.load(open("./data/Sort/n_vars-10_ascending-True_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
position = lab_syn["position"]
ascending = lab_syn["ascending"]
question = Sort.questions_syn[0].format(k=position, heavy = "heavy" if ascending else "light")
prompt_syn = Sort.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)
# Nat
# object_name = list(lab_nat[agent_name].keys())  # same...
# gt = lab_nat[agent_name][object_name]
question = Sort.questions_nat[0].format(k=position, biggest_smallest= "biggest" if ascending else "smallest")
prompt_nat = Sort.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Loops

In [ ]:
# Test the prompts
import json 
from codesim.prompt import Loops

# Get the data
data = json.load(open("./data/Loops/n_loops-5_n_noisy_loops-2_min_loop_length-3_max_loop-5_batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
first_key = list(lab_syn.keys())[0]
question = Loops.questions_syn[0].format(varname=first_key)
prompt_syn = Loops.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)
# Nat
key_val = list(lab_nat.keys())[0]
content, box = key_val.split(" in ")
# object_name = list(lab_nat[agent_name].keys())  # same...
# gt = lab_nat[agent_name][object_name]
question = Loops.questions_nat[0].format(varname=content, varname_origin=box)
prompt_nat = Loops.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

#### Boxes

In [ ]:
# Test the prompts
import json 
from codesim.prompt import Boxes

# Get the data
data = json.load(open("./data/boxes/batch-1.json"))
p_syn, p_nat, lab_syn, lab_nat = data[0].values()

# Syn
first_key = list(lab_syn.keys())[0]
question = Boxes.questions_syn[0].format(varname=f"x{first_key}")
prompt_syn = Boxes.user_cot.format(prefix="Here's some code:", 
                                problem=p_syn, 
                                question=question)
print(prompt_syn)

# Nat
first_key = list(lab_nat.keys())[0]
# object_name = list(lab_nat[agent_name].keys())  # same...
# gt = lab_nat[agent_name][object_name]
question = Boxes.questions_nat[0].format(varname=f"box {first_key}")
prompt_nat = Boxes.user_cot.format(prefix="",
                                problem=p_nat,
                                question=question)
print(prompt_nat)

### 3. Test exec-time SambaNova

In [ ]:
import os
import openai
import json
from tqdm import tqdm
from time import sleep
from codesim.prompt import StraightLine as SL

client = openai.OpenAI(
    api_key="380072e2-1a5e-4d75-b7a7-d657a457c416",
    base_url="https://api.sambanova.ai/v1",
)

def call(x:str) -> str:
    response = client.chat.completions.create(
        model='Meta-Llama-3.1-405B-Instruct',
        messages=[{"role":"system","content":"You are a helpful assistant"},{"role":"user","content":x}],
        temperature =  0.1,
        top_p = 0.1
    )

    return response.choices[0].message.content

# Get the data
data = json.load(open("./data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json"))

for idx, d in tqdm(enumerate(data)):
    p_syn, p_nat, lab_syn, lab_nat = d.values()
    var_name = list(lab_syn.keys())[0]  # can sample here
    gt = lab_syn[var_name]
    question = SL.questions_syn[0].format(varname=var_name)
    prompt_syn = SL.user_cot.format(prefix="Here's some code:", 
                                    problem=p_syn, 
                                    question=question)
    responses = {}
    try:
        responses[idx] = { "prompt": prompt_syn,
                           "response": call(prompt_syn),
                           "gt": gt
                        }
    except:
        sleep(5)
        
json.dump(responses, open("responses.json", "w"), indent=4)

### 1. Retrieve and save data

In [ ]:
from codesim.problem import StraightLine 

n_vars = 3; n_instances = 2
for n_ops in [10, 20, 30, 40, 50]:
    gen = StraightLine(n_ops=n_ops, 
                    n_vars=n_vars,
                    n_instances=n_instances)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()


In [ ]:
from codesim.problem import CriticalPath 

n_vars = 6; len_critical_path = 10
for n_ops in [10, 20, 30, 40, 50]:
    gen = CriticalPath(n_ops=n_ops, 
                       n_vars=n_vars,
                       len_critical_path=10)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()

In [ ]:
from codesim.problem import ParallelPaths 

n_vars = 3; n_instances = 2
for n_ops in [10, 20, 30, 40, 50]:
    gen = ParallelPaths(n_ops=n_ops, 
                        n_vars=n_vars,
                        n_instances=n_instances)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()

In [ ]:
from codesim.problem import Loops 

n_loops = [n for n in range(3, 11)]
n_noisy_loops = [int(n/2) for n in n_loops]
for n,m in zip(n_loops, n_noisy_loops):
    gen = Loops(n_loops=n, 
                n_noisy_loops=m,
                min_loop_length=2,
                max_loop_length=10)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()

In [ ]:
from codesim.problem import Sort 

for n_vars in [5, 10, 15, 20, 25]:
    # Ascending
    gen = Sort(n_vars=n_vars,
            ascending=True)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()
    
    # Descending    
    gen = Sort(n_vars=n_vars,
            ascending=False)

    # Generate and save 3 batches of 30 problems each
    for i in range(3):
        gen.generate_data(n_programs=30)
        gen.to_file(suffix="batch-" + str(i+1))
        gen.reset()
        
    

In [1]:
import sys
from codesim.kim_schuster import real_main
%load_ext autoreload
%autoreload 2

for n_items in [3, 4, 5, 6, 7]:
    # simulate CLI run
    sys.argv = ["codesim/kim_schuster.py", 
                "--max_items_per_box", f"{n_items}",
                "--num_samples", "30"]

    real_main()

Namespace(object_vocabulary_file='data/objects_with_bnc_frequency.csv', disjoint_object_vocabulary_file=None, disjoint_object_splits='test', output_dir='data/boxes', seed=2255, num_boxes=7, expected_num_items_per_box=1, max_items_per_box=3, num_samples=30, num_operations=10, disjoint_numops=False, zero_shot=False, include_modifiers='never', omit_modifiers_in_ops='never', alternative_forms='never', rarify=False, all_contents_operation=False)
finished sampling 60 sequences of length 10.
Saved outputs to data/boxes.
Namespace(object_vocabulary_file='data/objects_with_bnc_frequency.csv', disjoint_object_vocabulary_file=None, disjoint_object_splits='test', output_dir='data/boxes', seed=2255, num_boxes=7, expected_num_items_per_box=1, max_items_per_box=4, num_samples=30, num_operations=10, disjoint_numops=False, zero_shot=False, include_modifiers='never', omit_modifiers_in_ops='never', alternative_forms='never', rarify=False, all_contents_operation=False)
finished sampling 60 sequences of le

### 2. Generate prompts

#### Straight Line

### Starting the Tests for each dataset

In [ ]:
import sys
from codesim.experiment import main as start_experiment
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/experiment.py", 
            "--dataset_path", "data/boxes/batch-1.json",
            "--model", "gpt-4o-mini",
            "--operation", "kim-schuster"]

start_experiment()

In [ ]:
import sys
from codesim.experiment import main as start_experiment
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/experiment.py", 
            "--dataset_path", "data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json",
            "--model", "gpt-4o-mini",
            "--operation", "straight-line"]

start_experiment()

In [ ]:
import sys
from codesim.experiment import main as start_experiment
%load_ext autoreload
%autoreload 2
# simulate CLI run
sys.argv = ["codesim/experiment.py", 
            "--dataset_path", "data/StraightLine/n_ops-20_n_vars-3_n_instances-2_batch-1.json",
            "--model", "gpt-4o-mini",
            "--operation", "sorting"]

start_experiment()